In [51]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
np.set_printoptions(precision=3, linewidth=2*75, suppress=False)
from IPython.display import clear_output
from rise_and_fall import *
from dataset import Dataset
import json
import time

In [ ]:
def plot_frame(f:pd.DataFrame, mxis=[], mnis=[], mark=None):
    ps = f["meanp"].to_numpy()
    mn = ps.min()
    mx = ps.max()
    plt.plot(ps)
    if len(mxis)>0:
        plt.vlines(x = mxis, ymin = mn, ymax = mx, colors = 'green',)
    elif 'mxis' in f.columns:
        plt.vlines(x = f.index[f['mxis']>0], ymin = mn, ymax = mx, colors = 'green',)
    if len(mnis)>0:
        plt.vlines(x = mnis, ymin = mn, ymax = mx, colors = 'red',)
    elif 'mnis' in f.columns:
        plt.vlines(x = f.index[f['mnis']>0], ymin = mn, ymax = mx, colors = 'red',)
    if mark is not None:
        plt.vlines(x = [mark], ymin = mn, ymax = mx, colors = 'blue',)
    plt.show()
    
def unique(xs):
    ys = []
    for x in xs:
        if x in ys:
            continue
        ys.append(x)
    return ys

In [ ]:
initial_learning_rate = 0.01
decay_rate=0.5
decay_steps=5
y = lambda step: initial_learning_rate * (decay_rate ** (step / decay_steps))

steps = np.array([i for i in range(50)])
plt.plot(steps, y(steps))

In [63]:
def format_time(seconds):
    if seconds%60 == seconds:
        return f'{seconds} secs'
    else:
        return f'{np.floor(seconds/60.)} mins & {seconds%60.} secs'